In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import findspark
findspark.init()
import os
import pandas as pd
from pyspark.sql.functions import min, max, count, avg, col
from pyspark.sql.types import IntegerType, BooleanType, DateType
from pyspark.sql.window import Window
import csv 

# Part - 1 Task 1

In [2]:
url = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/groceries.csv'

In [3]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.addFile(url)

In [4]:
rd = spark.sparkContext.textFile(SparkFiles.get("groceries.csv")).map(lambda x: x.split(','))
rd.take(5)

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

# Part - 1 Task 2 - A

In [5]:
os.chdir(r'C:\Users\nares\Downloads')

In [6]:
unq_lst = rd.flatMap(lambda x: x).distinct().collect()
unq_lst = ['product '+i for i in unq_lst]
txtfile = open(file = 'out_1_2a.txt',mode = 'w')
for file in unq_lst:
    txtfile.write(file+'\n')
txtfile.close()

# Part - 1 Task 2 - B

In [7]:
totcnt = rd.flatMap(lambda x: x).count()
reccnt = open(file = 'out_1_2b.txt',mode = 'w')
reccnt.write('Count: \n'+str(totcnt))
reccnt.close()

# Part - 1 Task 3

In [8]:
sample = rd.flatMap(lambda x:x).map(lambda x:(x,1)). \
         reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1],ascending = False).take(5)
top_pur = open(file = 'out_1_3.txt',mode = 'w')
for file in sample:
    top_pur.write(str(file)+'\n')
top_pur.close()

# Part - 2 Task 1

In [9]:
df = spark.read.parquet('C:/Users/nares/Downloads/part-00000-tid-4320459746949313749-5c3d407c-c844-4016-97ad-2edec446aa62-6688-1-c000.snappy.parquet',header = True)
pd.set_option('display.max_columns',None)
df.toPandas()

,host_is_superhost,cancellation_policy,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na
0,t,moderate,t,1.0,Western Addition,37.76931,-122.43386,Apartment,Entire home/apt,3.0,1.0,1.0,2.0,Real Bed,1.0,180.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,170.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,f,strict_14_with_grace_period,f,2.0,Bernal Heights,37.74511,-122.42102,Apartment,Entire home/apt,5.0,1.0,2.0,3.0,Real Bed,30.0,111.0,98.0,10.0,10.0,10.0,10.0,10.0,9.0,235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,f,strict_14_with_grace_period,f,10.0,Haight Ashbury,37.76669,-122.45250,Apartment,Private room,2.0,4.0,1.0,1.0,Real Bed,32.0,17.0,85.0,8.0,8.0,9.0,9.0,9.0,8.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,f,strict_14_with_grace_period,f,10.0,Haight Ashbury,37.76487,-122.45183,Apartment,Private room,2.0,4.0,1.0,1.0,Real Bed,32.0,8.0,93.0,9.0,9.0,10.0,10.0,9.0,9.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,f,strict_14_with_grace_period,f,2.0,Western Addition,37.77525,-122.43637,House,Entire home/apt,5.0,1.5,2.0,2.0,Real Bed,7.0,27.0,97.0,10.0,10.0,10.0,10.0,10.0,9.0,785.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141,f,flexible,t,18.0,Noe Valley,37.74884,-122.42830,House,Entire home/apt,3.0,1.0,1.0,2.0,Real Bed,30.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,163.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7142,t,flexible,t,10.0,Russian Hill,37.80645,-122.42109,Guest suite,Entire home/apt,2.0,1.0,0.0,1.0,Real Bed,1.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,160.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7143,t,flexible,t,10.0,Western Addition,37.78855,-122.43110,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,2.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,249.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7144,f,flexible,t,87.0,Downtown/Civic Center,37.78645,-122.41458,Apartment,Entire home/apt,3.0,1.0,1.0,2.0,Real Bed,30.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,105.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Part - 2 Task 2

In [10]:
lst = df.select(min('price').cast(IntegerType()).alias('min_price'),max('price'). \
                cast(IntegerType()).alias('max_price'),count('price'). \
                alias('row_count')).rdd.flatMap(lambda x:x).collect()

header = ['min_price', 'max_price', 'row_count']
with open('out_2_2.csv', 'w',encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(lst)

# Part - 2 Task 3

In [11]:
from pyspark.sql.functions import avg, col
bbrm = df.filter((col('price') > 5000) & (col('review_scores_value') == 10) ) \
            .select(avg('bathrooms').alias('avg_bathrooms'),avg('bedrooms').alias('avg_bedrooms')) \
            .rdd.flatMap(lambda x:x).collect()
header = ['avg_bathrooms', 'avg_bedrooms']
with open('out_2_3.csv', 'w',encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(bbrm)

# Part - 2 Task 4

In [12]:
w1 = Window.orderBy('review_scores_accuracy_na')

acc = df.withColumn('highrating', max('review_scores_rating').over(w1)) \
    .withColumn('lowprice', min('price').over(w1)) \
    .filter((col('review_scores_rating') == col('highrating')) & (col('price') == col('lowprice'))) \
    .rdd.count()
reccnt = open(file = 'out_2_4.txt',mode = 'w')
reccnt.write(str(acc))
reccnt.close()